In [1]:
import os
import sys
sys.path.append('/home/kaustubh/Documents/GitHub/Self_Driving_Car/')

In [7]:
import torch
from stable_baselines3 import SAC
from carla_gym_env import CarlaEnv
from custom_feature_extractor import CustomFeatureExtractor
from stable_baselines3.common.callbacks import CheckpointCallback

In [ ]:
CHECKPOINTS_DIR = "checkpoints"
CHECKPOINT_PATH = os.path.join(CHECKPOINTS_DIR, "sac_carla_model")
TOTAL_TIMESTEPS = 5_000_000
SAVE_INTERVAL = 100_100

In [ ]:
def main():
    os.makedirs(CHECKPOINTS_DIR, exist_ok=True)
    
    env = CarlaEnv()
    policy_kwargs = {
        'features_extractor_class' : CustomFeatureExtractor,
        'features_extractor_kwargs' : {},
        'net_arch' : dict(pi=[768, 256, 256], qf=[768, 256, 256])
    }
    
    if os.path.exists(CHECKPOINT_PATH + ".zip"):
        model = SAC.load(CHECKPOINT_PATH, env=env, device="cuda" if torch.cuda.is_available() else 'cpu')
        print(f"Resuming training from checkpoint: {CHECKPOINT_PATH}")
    else:
        print("Starting new training.")
        
        model = SAC(
            "CnnPolicy",
            env,
            policy_kwargs=policy_kwargs,
            learning_rate=3e-4,
            buffer_size=10000,
            learning_starts=5000,
            batch_size=64,
            tau=0.005,
            gamma=0.99,
            train_freq=1,
            gradient_steps=1,
            verbose=1,
            device='cpu'
        )
    
    checkpoint_callback = CheckpointCallback(save_freq=SAVE_INTERVAL, save_path=CHECKPOINTS_DIR, name_prefix='sac_carla_model')
    model.learn(total_timesteps=TOTAL_TIMESTEPS, callback=checkpoint_callback, progress_bar=True)
    model.save("sac_carla_model")
    
    final_model_path = os.path.join(CHECKPOINTS_DIR, f"sac_final_{TOTAL_TIMESTEPS//1000}k")
    model.save(final_model_path)
    print(f"[DONE] Final model saved to: {final_model_path}")


In [ ]:
if __name__ == "__main__":
    main()

/home/kaustubh/miniconda3/envs/carla_15/lib/python3.10/site-packages/gymnasium/spaces/box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/home/kaustubh/miniconda3/envs/carla_15/lib/python3.10/site-packages/gymnasium/spaces/box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(
/home/kaustubh/miniconda3/envs/carla_15/lib/python3.10/site-packages/stable_baselines3/common/buffers.py:242: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 23.04GB > 22.95GB
  warnings.warn(


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/kaustubh/miniconda3/envs/carla_15/lib/python3.10/site-packages/torchvision/transforms/functional.py:154: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:203.)
  img = torch.from_numpy(pic.transpose((2, 0, 1))).contiguous()


Lane Invaded
